<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/DL/Group_3DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning with PyTorch
## 3D Convolutional Neural Network on Group Brain fMRI
Contributors: Stacey Rivet Beck, Ben Merrill
### To Do:
- Either:
  - Get Raw data in 4D   
          - OR -
  - Reshape Raw data into 4D from 2D
    - Apply Whole Brain Mask to data and save to AWS

- Build Dataloader: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

- Implement 3DCNN from paper: REALLY GREAT PAPERS
  - Nguyen et al. http://proceedings.mlr.press/v136/nguyen20a/nguyen20a.pdf
  - Wang et al. https://arxiv.org/pdf/1801.09858.pdf (discusses more in detail the input data shapes and processing)
  
  - Inputs: 84@ x * y * z ; one fmri time series at a time, not concatenated
  - Basic Architecture:
        #First layer is generating temporal descriptors of the voxels
        Conv1 1 x 1 x 1 filter, output = 32, stride = 1, ReLU, BatchNorm
        Conv2 7 x 7 x 7 filter, output = 64, stride = 2, ReLU, BatchNorm
        Conv3 3 x 3 x 3 fitler, output = 64, stride = 2, ReLU, BatchNorm
        Conv4 3 x 3 x 3 fitler, output = 128, stride = 2, ReLU, BatchNorm
        Global Average Pooling on final feature maps ->
        Flattened maps size 128?
        Fully connected layer size 64
        Fully connected layer size 2 (2 way classification, one for each class) -> softmax

        Optimized with Adam, standard parameters (β1=0.9 and β2=0.999)
        Batched at 32, but we may need to batch smaller due to GPU compute
        Learning Rate = 0.001, gradual decay after Val loss plateaued after 15 epochs
        Cross entropy Loss
        Employ early stopping
        In Wang et al. they used data for visualization, same size as input data, though are reduced in time dimension to be mapped on fsaverage surface. 
        
  

## Importing Dataset and Labels

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')  

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git
# Change directory into cloned repo
%cd teambrainiac/source/DL
!ls

Cloning into 'teambrainiac'...
remote: Enumerating objects: 909, done.
remote: Counting objects: 100% (909/909), done.
remote: Compressing objects: 100% (676/676), done.
remote: Total 909 (delta 572), reused 428 (delta 217), pack-reused 0
Receiving objects: 100% (909/909), 75.12 MiB | 24.01 MiB/s, done.
Resolving deltas: 100% (572/572), done.
/content/teambrainiac/source/teambrainiac/source/DL/teambrainiac/source/DL
Group_3DCNN.ipynb  process_dl.py  utils_dl.py


### Load path_config.py

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 196 bytes


## Import Packages

In [ ]:
# General Library Imports
import re
import scipy.io
import os
import pickle
import numpy as np
import nibabel as nib
import pandas as pd
import boto3
import tempfile
import tqdm
from path_config import mat_path
from botocore.exceptions import ClientError
from collections import defaultdict

# From Local Directory
from utils_dl import *
from process_dl import *

# Pytroch Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

#import torchvision.transforms as transforms
from torch.nn import ReLU, CrossEntropyLoss, Conv3d, Module, Softmax, AdaptiveAvgPool3d
from torch.optim import Adam, SGD

#from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

## Import Group fMRI Data, Normalize, and Create Masks

In [ ]:
# Get Mask and Open data_path_dict

# def open_pickle(file_path):
#     """
#     :param file path for dictionary
#     :return dictionary:
#     """
#     f = open(file_path, "rb")
#     dictionary = pickle.load(f)
#     f.close()

#     return dictionary


# def get_mask(mask_type,data_path_dict,mask_ind):
#   """
#     Function to return the mask of what brain voxels we want to include in analysis
#     Params:
#       data_path_dict  : dictionary containing paths to all data stored on AWS
#       mask_type: name of mask we want to use
#       mask_ind: index of where the path to the masks are 0: full brain mask plus masks that subtract region
#               1: Regions of interest(ROIs) mask out full brain except structure we care about
#   """
#   mask_data_filepath = data_path_dict['mask_data'][mask_ind] #path to masked data     
#   mask_type_dict = access_load_data(mask_data_filepath, True) #get the mask data dictionary
#   np_array_mask = mask_type_dict[mask_type] #get the mask array
#   mask = np.ma.make_mask(np_array_mask).reshape(79*95*79,order='F') #create a 1-D array for the mask. Important to use Fourier Transformation as we are working in brain space!

#   return mask

# # def access_aws():
# #     """
# #     :return:
# #     """

# #     # Acces AWS S3 MATLAB file
# #     pubkey = mat_path['ACCESS_KEY']
# #     seckey = mat_path['SECRET_KEY']
# #     client = boto3.client('s3', aws_access_key_id = pubkey, aws_secret_access_key = seckey)
# #     s3 = boto3.resource('s3', aws_access_key_id = pubkey, aws_secret_access_key = seckey)
# #     bucket = s3.Bucket('teambrainiac')
# #     bucket_ = bucket.name
# #     obj_name = list(bucket.objects.all())

# #     return obj_name, bucket_, client



# # def load_mat(path):
# #     """
# #     :param .mat data path
# #     uses scipy to convert access to mat file in python
# #     :return mat data
# #     """
# #     mat_file = scipy.io.loadmat(path)
# #     return mat_file




# def labels_mask_binary(data_path_dict, label_type='rt_labels'):
#     """
#     """
#     label_path = data_path_dict['labels'][0]
#     label_data_dict = access_load_data(label_path, True)
#     labels = label_data_dict[label_type].T[0]
#     image_label_mask = np.array([bool(x!=9999) for x in labels])
#     image_labels = labels[image_label_mask]

#     return image_label_mask, image_labels





# def access_load_data(obj, bool_mat):
#     """
#     :param data_file: file path to data. For example can be pickle file containing a dictionary
#                       or csv file path to load as a dataframe, e.g. file_names_dict['subject_data'][0]
#                       or nifti file path to load as nifti object
#                       or load a matlab file if bool_mat == TRUE
#     :param bool_mat:  if true will run load_mat() and return .mat file
#     :return        :  will open matlab data if bool_mat == True,
#                       pickle file
#                       csv file as a dataframe
#                       nifti file
#     """

#     # Connect to AWS client
#     _, bucket_, client = access_aws()

#     # Create a temporary file
#     temp = tempfile.NamedTemporaryFile()

#     # Download data in temp file
#     client.download_file(bucket_, obj, temp.name)

#     if bool_mat == True:
#         data = load_mat(temp.name)
#     else:
#         if '.pkl' in obj:
#             data = open_pickle(temp.name)
#         elif '.csv' in obj:
#             data = pd.read_csv(temp.name)
#         elif '.nii' in obj:
#             temp_f = 'data/temp.nii'
#             client.download_file(bucket_, obj, temp_f)
#             data = data_to_nib(temp_f)

#     temp.close()

#     return data



# def load_subjects_chronologically(data_path_dict, n_subjects, image_label_mask, image_labels, label_type='rt_labels', runs=[2, 3]):
#   '''
#     Function to load subject data. This deletes images with no labels and returns only the runs of interest for each subject.
#     Params:
#       data_path_dict  : dictionary that has paths to data on AWS
#       n_subjects      : the number of subjects
#       image_label_mask: a mask indicating whether a binary label exists for each image in a run
#       image_labels    : binary labels indicating whether a subject was up or down regulating 
#       label_type      : the type of label to return from the labels file in AWS 
#       runs            : a list of runs to return from each subject

#     returns: dictionary of users and their runs
#   '''
#   # Load subject Ids
#   subject_paths = data_path_dict['subject_data'][:n_subjects]
#   subject_ids = data_path_dict['subject_ID'][:n_subjects]
#   subjects = {}

#   for path,id in tqdm.tqdm(zip(subject_paths, subject_ids)):
#     subject_dict = {}
#     data = access_load_data(path,True)
#     subject_dict['image_labels'] = image_labels

#     for run in runs:
#       run_key = 'run_0' + str(run) + '_vec'
#       run_masked = data[run_key][image_label_mask]
#       subject_dict['run_'+str(run)] = run_masked
    
#     subjects[id] = subject_dict
  
#   return subjects

In [ ]:
# Open path dictionary file to get subject ids
path = "../data/data_path_dictionary.pkl"
data_path_dict = open_pickle(path)

NameError: ignored

In [ ]:
# Fix hyperparameters for preprocessing
label_type='rt_labels'
n_subjects = 5
runs_list = [2]


# Run functions to import unmasked data
image_label_mask, image_labels = labels_mask_binary(data_path_dict, label_type='rt_labels')

five_subjects_run_2 = load_subjects_chronologically(data_path_dict, n_subjects, image_label_mask, image_labels, label_type, runs_list)

5it [00:58, 11.71s/it]


In [ ]:
five_subjects_run_2.keys()

dict_keys(['10004_08693', '10008_09924', '10009_08848', '10016_09694', '10017_08894'])

In [ ]:

five_subjects_run_2['10004_08693']


{'image_labels': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], dtype=uint16),
 'run_2': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int16)}

In [ ]:
# Get Mask Indicies. Note that 'mask' is the string for a whole brain mask.
whole_mask = get_mask('mask', data_path_dict, mask_ind=0)
whole_mask


array([False, False, False, ..., False, False, False])

## Build Dataloader

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

## Build Model

In [ ]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    
    #Conv1
    self.conv1 = nn.Conv3d(in_channels = 1, 
                           out_channels = 32, 
                           kernel_size = 1, 
                           stride = 1
                           )
    self.bn1 = nn.BatchNorm3d(32)
    self.conv2 = nn.Conv3d(in_channels = 32, 
                           out_channels = 64, 
                           kernel_size = 7, 
                           stride = 2
                           )
    self.bn2 = nn.BatchNorm3d(64)
    self.conv3 = nn.Conv3d(in_channels = 64, 
                           out_channels = 64, 
                           kernel_size = 3, 
                           stride = 2
                           )
    self.bn3 = nn.BatchNorm3d(64)
    self.conv4 = nn.Conv3d(in_channels = 64, 
                           out_channels = 128, 
                           kernel_size = 3, 
                           stride = 2
                           )
    self.bn4 = nn.BatchNorm3d(128) 
    self.pool1 = nn.AdaptiveAvgPool3d((1,1,1)) #Global Average Pool, takes the average over last two dimensions to flatten 
  
                                                             
    # Fully connected layer
    self.fc1 = nn.Linear(128*?*?,64) # need to find out the size where AdaptiveAvgPool 
    self.fc2 = nn.Linear(64, 2) # left with 2 for the two classes                     



  def forward(self, x):
    x = self.bn1(F.relu(self.conv1((x))))
    x = self.bn2(F.relu(self.conv2((x))))
    x = self.bn3(F.relu(self.conv3((x))))
    x = self.pool1(self.bn4(F.relu(self.conv4((x)))))
    x = self.fc1(x)
    x = F.softmax(self.fc2(x))
    return x                        




# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Get model
model = ConvNet()
model = model.to(device)
print("First model training on GPU")
print(model)

# Initialize other parameters
epochs = 25 #120
learning_rate = 0.001
criterion = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum

## Training

In [ ]:
accuracy_stats = {
    'train': [],
    'val': []
  }

print(accuracy_stats)

loss_stats = {
    'train': [],
    'val': []
    }
print(loss_stats)



def train_val_model(epochs):
  for epoch in range(1, epochs + 1):

    # TRAINING *****************************************************************

    train_epoch_loss = 0
    train_epoch_acc = 0

    # set model in training mode 
    model.train()
    print('\nEpoch$ : %d'%epoch)
    for x_train_batch, y_train_batch in tqdm(train_loader):
      x_train_batch = x_train_batch.to(device)#(float).to(device) # for GPU support
      y_train_batch = y_train_batch.to(device) 

      #print(x_train_batch.shape)

      # sets gradients to 0 to prevent interference with previous epoch
      optimizer.zero_grad()
    
      # Forward pass through NN
      y_train_pred = model(x_train_batch)#.to(float)
      train_loss = criterion(y_train_pred, y_train_batch)
      train_acc = accuracy(y_train_pred, y_train_batch)

      # Backward pass, updating weights
      train_loss.backward()
      optimizer.step()

      # Statistics
      train_epoch_loss += train_loss.item()
      train_epoch_acc += train_acc.item()


    # VALIDATION****************************************************************   
    
    with torch.set_grad_enabled(False):
      val_epoch_loss = 0
      val_epoch_acc = 0

      model.eval()
      for x_val_batch, y_val_batch in tqdm(validate_loader):
      
        x_val_batch =  x_val_batch.to(device)#.to(float)
        y_val_batch = y_val_batch.to(device)
            
        # Forward pass
        y_val_pred = model(x_val_batch)#.to(float)   
        val_loss = criterion(y_val_pred, y_val_batch)
        val_acc = accuracy(y_val_pred, y_val_batch)
            
        val_epoch_loss += val_loss.item()
        val_epoch_acc += val_acc.item()

    # Prevent plateauing validation loss 
    #scheduler.step(val_epoch_loss/len(validate_loader))

        
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(validate_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(validate_loader))
                              
    
    print(f'Epoch {epoch+0:03}: Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(validate_loader):.5f}') 
    print(f'Train Acc: {train_epoch_acc/len(train_loader):.3f} | Val Acc: {val_epoch_acc/len(validate_loader):.3f}')

      




def accuracy(y_pred, y_test):
  # Calculating model accuracy at each epoch 
  y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
  _, y_pred_prob = torch.max(y_pred_softmax, dim = 1)
  correct_pred = (y_pred_prob == y_test).float()
  acc = correct_pred.sum() / len(correct_pred)
  acc = torch.round(acc * 100)

  return acc



     





if __name__ == '__main__':
  train_val_model(epochs)

In [ ]:
def open_pickle(file_path):
    """
    :param file path for dictionary
    :return dictionary:
    """
    f = open(file_path, "rb")
    dictionary = pickle.load(f)
    f.close()

    return dictionary






def access_load_data(obj, bool_mat):
    """
    :param data_file: file path to data. For example can be pickle file containing a dictionary
                      or csv file path to load as a dataframe, e.g. file_names_dict['subject_data'][0]
                      or nifti file path to load as nifti object
                      or load a matlab file if bool_mat == TRUE
    :param bool_mat:  if true will run load_mat() and return .mat file
    :return        :  will open matlab data if bool_mat == True,
                      pickle file
                      csv file as a dataframe
                      nifti file
    """

    # Connect to AWS client
    _, bucket_, client = access_aws()

    # Create a temporary file
    temp = tempfile.NamedTemporaryFile()

    # Download data in temp file
    client.download_file(bucket_, obj, temp.name)

    if bool_mat == True:
        data = load_mat(temp.name)
    else:
        if '.pkl' in obj:
            data = open_pickle(temp.name)
        elif '.csv' in obj:
            data = pd.read_csv(temp.name)
        elif '.nii' in obj:
            temp_f = 'data/temp.nii'
            client.download_file(bucket_, obj, temp_f)
            data = data_to_nib(temp_f)

    temp.close()

    return data






def s3_upload(data, object_name, data_type):
    """Upload a file to an S3 bucket
    :param data: our data to upload
    :param data_type: type of data file we are creating
    :param object_name: S3 object name. If not specified then name of temp.name is used
    :return: True if file was uploaded, else False
    """

    # Upload the file
    # Connect to AWS client
    _, bucket_name, client = access_aws()

    try:

        with tempfile.NamedTemporaryFile(delete=False) as temp:
            if data_type == "pickle":
                pickle.dump(data, temp, protocol=pickle.HIGHEST_PROTOCOL)

            elif data_type == "numpy":
                np.save(temp, data)
                _ = temp.seek(0)

            elif data_type == "csv":
                data.to_csv(temp, index=False)

            client.upload_file(temp.name, bucket_name, object_name)
            temp.close()
            print(f"upload complete for {object_name}")

        if data_type == "nifti":
            tempf = 'data/upload_temp.nii'
            nib.save(data, tempf)
            client.upload_file(tempf, bucket_name, object_name)
            print(f"upload complete for {object_name}")

    except ClientError as e:
        logging.error(e)
        return False


    return True





def load_mask_indices(data_paths, mask_type, m_path_ind):
    """
    :param data_paths:
    :param mask_type:
    :param m_path_ind:
    :return:
    """
    mask_data_path = data_paths['mask_data'][m_path_ind]
    mask_type_dict = access_load_data(mask_data_path, True)
    np_array_mask = mask_type_dict[mask_type]
    print("mask shape:", np_array_mask.shape)
    np_compatible_mask = np.ma.make_mask(np_array_mask).reshape(79 * 95 * 79, order='F')
    indices_mask = np.where(
        np_compatible_mask == 1)  # gets the indices where the mask is 1, the brain region for x, y, z planes

    return indices_mask